<a href="https://colab.research.google.com/github/TraverseTowner/my_dataset/blob/master/BosaiModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from keras.models import Sequential
# from keras.callbacks import EarlyStopping
# from keras.preprocessing import sequence
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dropout, Dense, Activation, LSTM
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import backend as K 

import numpy as np
import sqlite3
from math import floor
from typing import List
from typing import Sequence
import time
from tensorflow.keras.utils import to_categorical   

import os
import tensorflow as tf
# tf.test.gpu_device_name()

In [0]:
VOCAB_SIZE = 75313 + 1
EMBEDDING_SIZE = 50
INPUT_SIZE = 8
OUTPUT_SIZE = 4635

def __RNN():
        model = Sequential()
#         model.add(tf.keras.layers.Dense(5, input_shape=(5,), activation=tf.nn.relu)) 
        model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, input_length=INPUT_SIZE, input_shape=(VOCAB_SIZE, )))
        model.add(SpatialDropout1D(0.2))
        model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(OUTPUT_SIZE, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
        return model


def __one_hot(arr: np.ndarray, num_classes: int) -> np.ndarray:
        """
        Obtain the one-hot representation from an one-dimensional class.
        :param a: One-dimensional array that contains the classes (e.g. [1,1,...,6])
        :param num_classes: The total number of classes.
        :return 2d_np_array:
        """

        res = np.zeros((arr.size, num_classes), dtype='int')

        for i, group in enumerate(arr):
            # The group begins from 1, and hence we must subtact by one.
            res[i][group-1] = 1
        return res

      

      


      

def train_model(features_dir: str, target_dir: str, save_file_name: str):

#         my_model = __RNN()
#         # model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
#         print(my_model.summary())
        
        TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
        tf.logging.set_verbosity(tf.logging.INFO)
        
#         resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#         tf.config.experimental_connect_to_host(resolver.master())
#         tf.tpu.experimental.initialize_tpu_system(resolver)
#         strategy = tf.distribute.experimental.TPUStrategy(resolver)
        
        
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_host(resolver.master())
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)

#         resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
#         tf.contrib.distribute.initialize_tpu_system(resolver)
#         strategy = tf.contrib.distribute.TPUStrategy(resolver)

        TOTAL_NUMBER_OF_FILES_TO_COMPUTE = 10
        CHUNK_SIZE = 20000
        number_of_rows_to_skip = 0
        
    
        with strategy.scope():

            my_model = __RNN()
            # model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
            print(my_model.summary())
#             tpu_model = tf.contrib.tpu.keras_to_tpu_model(my_model, strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

            for file_id in range(TOTAL_NUMBER_OF_FILES_TO_COMPUTE):
              
              for _ in range(int(500000/CHUNK_SIZE)):

                X = np.loadtxt(features_dir + '/' + str(file_id) + '.txt', delimiter=',', dtype='int', skiprows = number_of_rows_to_skip, max_rows=CHUNK_SIZE)  
                t = np.loadtxt(target_dir + '/' + str(file_id) + '.txt', delimiter=',', dtype='int', skiprows = number_of_rows_to_skip, max_rows=CHUNK_SIZE)

                number_of_rows_to_skip += CHUNK_SIZE

                T = to_categorical(t, num_classes=OUTPUT_SIZE)

                print("X.shape", X.shape)
                print("T.shape", T.shape)

                X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.10, random_state=42)

                my_model.fit(X_train,
                            T_train,
                            batch_size=128,
                            epochs=5,
                            steps_per_epoch = 140,
                            validation_data=(X_test, T_test),
                            callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

                print(str(number_of_rows_to_skip))
                  
                if number_of_rows_to_skip == 500000:
                    print("FINISHED PROCESSING FILE " + str(file_id))
                    my_model.save('my_model_weights_' + str(file_id))
                    number_of_rows_to_skip = 0

# train_model("my_dataset/feature_5000_X", "my_dataset/output_5000_t", "my_awesome_model")

W0621 15:55:33.654155 140225416955776 tpu_strategy_util.py:57] TPU system %s has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.
I0621 15:55:33.656020 140225416955776 tpu_strategy_util.py:61] Initializing the TPU system: 10.91.167.194:8470
I0621 15:55:41.125016 140225416955776 tpu_strategy_util.py:102] Finished initializing TPU system.
I0621 15:55:41.127674 140225416955776 tpu_system_metadata.py:78] Querying Tensorflow master (grpc://10.91.167.194:8470) for TPU system metadata.
I0621 15:55:41.142316 140225416955776 tpu_system_metadata.py:148] Found TPU system:
I0621 15:55:41.144161 140225416955776 tpu_system_metadata.py:149] *** Num TPU Cores: 8
I0621 15:55:41.145426 140225416955776 tpu_system_metadata.py:150] *** Num TPU Workers: 1
I0621 15:55:41.146566 140225416955776 tpu_system_metadata.py:152] *** Num TPU Cores Per Worker: 8
I0621 15:55:41.147750 140225416955776 tpu_system_metadata.py:154] *** Available Device: _DeviceAttri

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 75314, 50)         3765700   
_________________________________________________________________
spatial_dropout1d_10 (Spatia (None, 75314, 50)         0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               60400     
_________________________________________________________________
dense_10 (Dense)             (None, 4635)              468135    
Total params: 4,294,235
Trainable params: 4,294,235
Non-trainable params: 0
_________________________________________________________________
None
X.shape (20000, 8)
T.shape (20000, 4635)
Epoch 1/5
136/140 [============================>.] - ETA: 0s - loss: nan - acc: 0.0000e+00

I0621 15:56:45.036142 140225416955776 training_distributed.py:262] Running validation at fit epoch: 0


140/140 [==============================] - 26s 185ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
20000
X.shape (20000, 8)
T.shape (20000, 4635)
Epoch 1/5
136/140 [============================>.] - ETA: 0s - loss: nan - acc: 0.0000e+00

I0621 15:58:10.077040 140225416955776 training_distributed.py:262] Running validation at fit epoch: 0


140/140 [==============================] - 31s 219ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
40000
X.shape (20000, 8)
T.shape (20000, 4635)
Epoch 1/5
139/140 [============================>.] - ETA: 0s - loss: nan - acc: 0.0000e+00

I0621 15:59:51.617652 140225416955776 training_distributed.py:262] Running validation at fit epoch: 0


KeyboardInterrupt: ignored

In [0]:
VOCAB_SIZE = 75313 + 1
EMBEDDING_SIZE = 50
INPUT_SIZE = 8
OUTPUT_SIZE = 4635

def __RNN():
        model = Sequential()
#         model.add(tf.keras.layers.Dense(5, input_shape=(5,), activation=tf.nn.relu)) 
        model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE, input_length=INPUT_SIZE))
        model.add(SpatialDropout1D(0.2))
        model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(OUTPUT_SIZE, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
        return model


def __one_hot(arr: np.ndarray, num_classes: int) -> np.ndarray:
        """
        Obtain the one-hot representation from an one-dimensional class.
        :param a: One-dimensional array that contains the classes (e.g. [1,1,...,6])
        :param num_classes: The total number of classes.
        :return 2d_np_array:
        """

        res = np.zeros((arr.size, num_classes), dtype='int')

        for i, group in enumerate(arr):
            # The group begins from 1, and hence we must subtact by one.
            res[i][group-1] = 1
        return res


def train_model_gpu(features_dir: str, target_dir: str, save_file_name: str):

        TOTAL_NUMBER_OF_FILES_TO_COMPUTE = 1
        CHUNK_SIZE = 100000
        number_of_rows_to_skip = 0
        my_model = __RNN()

        print(my_model.summary())

        for file_id in range(TOTAL_NUMBER_OF_FILES_TO_COMPUTE):

          start_time = time.time()
          for _ in range(int(500000/CHUNK_SIZE)):

            X = np.loadtxt(features_dir + '/' + str(file_id) + '.txt', delimiter=',', dtype='int', skiprows = number_of_rows_to_skip, max_rows=CHUNK_SIZE)  
            t = np.loadtxt(target_dir + '/' + str(file_id) + '.txt', delimiter=',', dtype='int', skiprows = number_of_rows_to_skip, max_rows=CHUNK_SIZE)

            number_of_rows_to_skip += CHUNK_SIZE

#             T = to_categorical(t, num_classes=OUTPUT_SIZE)
            T = __one_hot(t, num_classes=OUTPUT_SIZE)

#             print("X.shape", X.shape)
#             print("T.shape", T.shape)

            my_model.fit(X,
                       T,
                        batch_size=128,
                        epochs=5,
                        # validation_data=(X_test, T_test),
                        validation_split=0.1,
                        callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

            print("Number of Rows Parsed:", str(number_of_rows_to_skip))
            elapsed_time = time.time() - start_time
              
            if number_of_rows_to_skip == 500000:
                print("Finished Processing File:" + str(file_id))
                print("🎉 Took", floor(elapsed_time / 60), "minutes", round(elapsed_time % 60), "seconds")
                my_model.save('my_model_weights_' + str(file_id))
                number_of_rows_to_skip = 0

In [0]:
train_model_gpu("my_dataset/feature_5000_X", "my_dataset/output_5000_t", "my_awesome_model")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8, 50)             3765700   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 8, 50)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 4635)              468135    
Total params: 4,294,235
Trainable params: 4,294,235
Non-trainable params: 0
_________________________________________________________________
None
Train on 90000 samples, validate on 10000 samples
Epoch 1/5
90000/90000 [==============================] - 24s 263us/sample - loss: 3.1382 - acc: 0.0552 - val_loss: 15.4701 - val_acc: 0.0000e+00
Epoch 2/5
90000/90000 [===================

In [0]:
def push_to_git():
  !ls

In [0]:
push_to_git()

my_dataset  my_model_weights_0


In [0]:
def init_git():
  !git init
  !git config --global user.email "isshinada@gmail.com"
  !git remote add origin https://github.com/TraverseTowner/my_dataset.git
#   !git remote set-url origin git@github.com:TraverseTowner/my_dataset.git
def lol_cake(file_name):
  !git add $file_name
  !git commit -m "msg"
  !git push origin master

In [36]:
init_git()
lol_cake("my_model_weights_0")


OpenSSH_7.6p1 Ubuntu-4ubuntu0.3, OpenSSL 1.0.2n  7 Dec 2017
debug1: Reading configuration data /etc/ssh/ssh_config
debug1: /etc/ssh/ssh_config line 19: Applying options for *
debug2: resolving "github.com" port 22
debug2: ssh_connect_direct: needpriv 0
debug1: Connecting to github.com [13.229.188.59] port 22.
debug1: Connection established.
debug1: permanently_set_uid: 0/0
debug1: key_load_public: No such file or directory
debug1: identity file /root/.ssh/id_rsa type -1
debug1: key_load_public: No such file or directory
debug1: identity file /root/.ssh/id_rsa-cert type -1
debug1: key_load_public: No such file or directory
debug1: identity file /root/.ssh/id_dsa type -1
debug1: key_load_public: No such file or directory
debug1: identity file /root/.ssh/id_dsa-cert type -1
debug1: key_load_public: No such file or directory
debug1: identity file /root/.ssh/id_ecdsa type -1
debug1: key_load_public: No such file or directory
debug1: identity file /root/.ssh/id_ecdsa-cert type -1
debug1: key

In [0]:
!git -v

unknown option: -v
usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]


In [29]:
!ls

my_dataset  my_model_weights_0


In [0]:
!rm -r my_dataset
!git clone https://github.com/TraverseTowner/my_dataset.git

Cloning into 'my_dataset'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 135 (delta 2), reused 12 (delta 2), pack-reused 123
Receiving objects: 100% (135/135), 264.03 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [0]:
# !ls
!rm -r my_dataset
!rm -r sample_data

rm: cannot remove 'my_dataset': No such file or directory


In [0]:
!pip install -q tf-nightly-gpu


     |████████████████████████████████| 394.5MB 43kB/s 
     |████████████████████████████████| 501kB 45.4MB/s 
     |████████████████████████████████| 3.2MB 31.1MB/s 


my_dataset  sample_data


In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()